# On-Target Threshold
Determine the value of hyperparameter `cnfg.ON_TARGET_THRESHOLD`, which is used to decide if a gaze sample / fixation / visit is "on-target" or not.

In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

import config as cnfg

# pio.renderers.default = "notebook"
pio.renderers.default = "browser"

### Read data

In [2]:
from analysis.pipeline.full_pipeline import read_saved_data

_targets, _actions, _metadata, idents, fixations, _visits = read_saved_data()

### Show Data:
#### (1) Distances-from-Target when subject performed the identification action

In [3]:
percentiles = [0.5, 0.75, 0.85, 0.9, 0.95, 0.99,]

not_misses = idents.loc[idents["identification_category"] != "miss"]
dist_summary = (
    pd.concat([
        not_misses["distance_dva"].describe(percentiles).rename("all"),
        not_misses.groupby("subject")["distance_dva"].describe(percentiles).T,
    ], axis=1)
).T

dist_summary

,count,mean,std,min,50%,75%,85%,90%,95%,99%,max
all,1428.0,0.805839,1.613045,0.005250,0.531129,0.826878,1.005759,1.164547,1.415456,8.209510,23.297429
1,70.0,0.865243,2.177789,0.057528,0.533569,0.739879,0.843786,0.921057,0.956646,8.963045,18.223686
2,112.0,0.887389,1.378940,0.005250,0.638451,0.945330,1.132480,1.240759,1.367437,8.195211,9.435844
3,110.0,0.522337,0.318907,0.016496,0.446273,0.649872,0.818994,0.953997,1.151773,1.507079,1.723973
12,111.0,1.377363,2.263406,0.160329,0.863651,1.245024,1.640418,1.947023,4.304638,9.813110,19.109826
13,99.0,0.782881,1.451985,0.073701,0.538058,0.731862,0.912429,1.155491,1.408013,6.124180,13.597367
14,104.0,1.307629,3.840592,0.032188,0.370250,0.633209,0.733201,0.991192,5.101486,21.035096,23.297429
15,98.0,0.809266,1.268136,0.085205,0.675779,0.865602,1.059637,1.202942,1.377850,2.032327,12.765376
16,90.0,0.423305,0.247078,0.043483,0.350909,0.640706,0.747622,0.778273,0.858737,0.915361,0.967981
17,101.0,0.764956,1.272487,0.065453,0.567267,0.825397,1.034370,1.153490,1.318819,3.755980,12.600789


In [4]:
not_misses

,subject,trial,target,identification_category,time,to_trial_end,distance_px,distance_dva,left_x,left_y,left_pupil,right_x,right_y,right_pupil
0,1,1,target0,hit,8662.0,6197.0,16.394244,0.440860,794.7264,199.5624,2.74249,803.8848,182.6496,2.89998
2,1,2,target0,hit,5620.0,13270.0,31.469873,0.846260,844.5696,380.1708,2.38924,827.5776,401.4900,2.50285
4,1,3,target0,hit,9990.0,6202.0,34.602952,0.930512,1157.0880,240.7644,2.64009,1149.0240,250.1712,2.87772
9,1,5,target1,hit,9062.0,6569.0,8.794686,0.236499,NaN,NaN,NaN,413.4336,748.9908,2.55870
10,1,6,target0,hit,7780.0,9929.0,27.876230,0.749623,486.3936,253.6704,2.56209,462.7392,241.1748,2.70834
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2057,22,58,target0,hit,1544.0,12038.0,35.367994,0.935233,920.0256,478.3428,3.49547,865.8816,459.0000,3.67410
2058,22,58,target1,false_alarm,5160.0,8422.0,38.883752,1.028200,1391.4432,707.0760,3.74825,1366.4832,687.7440,3.87274
2060,22,59,target0,false_alarm,3408.0,7086.0,38.162648,1.009132,463.6224,644.8032,3.93939,428.2560,616.1616,4.05275
2062,22,59,target1,false_alarm,1640.0,8854.0,61.099378,1.615647,1018.0992,657.9900,3.45514,954.6624,636.3252,3.56114


In [5]:
fig = make_subplots(
    rows=2, cols=1, shared_xaxes=True, shared_yaxes=False,
)

# top: distribution across all subjects
fig.add_trace(
    row=1, col=1, trace=go.Violin(
        y0="distance", x=not_misses["distance_dva"],
        name="All Subjects", legendgroup="All Subjects",
        text=not_misses.apply(
            lambda row: f"Subject: {row['subject']}<br>"
                        f"Trial: {row['trial']}<br>"
                        # f"Target: {row['target']}<br>"
                        f"Distance: {row['distance_dva']:.2f} DVA",
            axis=1
        ),
        marker=dict(color=cnfg.get_discrete_color("all")),
        width=1.75, orientation="h", side="positive", spanmode='hard',
        box=dict(visible=False),
        meanline=dict(visible=True),
        points="all", pointpos=-0.5,
        showlegend=True, hoverinfo="x+y+text",

    )
)

# bottom: distribution per subject
for subj_id in not_misses[cnfg.SUBJECT_STR].unique():
    subj_string = f"{cnfg.SUBJECT_STR.capitalize()} {subj_id:02d}"
    subj_data = not_misses[not_misses[cnfg.SUBJECT_STR] == subj_id]
    texts = subj_data.apply(
        lambda row: f"{subj_string}<br>"
                    f"Trial: {row['trial']}<br>"
                    # f"Target: {row['target']}<br>"
                    f"Distance: {row['distance_dva']:.2f} DVA",
        axis=1
    )
    fig.add_trace(
        row=2, col=1, trace=go.Violin(
            y0="distance", x=subj_data["distance_dva"],
            text=texts,
            name=subj_string, legendgroup=subj_string,
            marker=dict(color=cnfg.get_discrete_color(subj_id, loop=True), opacity=0.5),
            width=1.75, orientation="h", side="positive", spanmode='hard',
            box=dict(visible=False),
            meanline=dict(visible=True),
            points="all", pointpos=-0.5,
            showlegend=True, hoverinfo="x+y+text"
        )
    )

# update visuals
fig.update_annotations(font=cnfg.AXIS_LABEL_FONT)
fig.update_yaxes(showticklabels=False)  # Hide y-axis labels
fig.update_xaxes(
    title=None, showline=False,
    showgrid=True, gridcolor=cnfg.GRID_LINE_COLOR, gridwidth=cnfg.GRID_LINE_WIDTH,
    zeroline=False, zerolinecolor=cnfg.GRID_LINE_COLOR, zerolinewidth=cnfg.ZERO_LINE_WIDTH,
    tickfont=cnfg.AXIS_TICK_FONT,
)
fig.update_layout(
    width=1200, height=675,
    title=dict(text="Distance on Identification-Action", font=cnfg.TITLE_FONT),
    paper_bgcolor='rgba(0, 0, 0, 0)',
    # plot_bgcolor='rgba(0, 0, 0, 0)',
    showlegend=True,
)

fig.show()

#### (2) Distances-from-Target across all fixations
#### (2) Distances-from-Target during identification-fixations

In [6]:
percentiles = [0.05, 0.25, 0.5, 0.75, 0.9, 0.95]

dva_cols = [col for col in fixations.columns if col.endswith("distance_dva")]
min_dists = pd.concat([fixations[["subject", "trial", "eye", "event"]], fixations[dva_cols].min(axis=1).rename("distance")], axis=1)
fixation_dist_summary = (
    pd.concat([
        min_dists["distance"].describe(percentiles).rename("all"),
        min_dists.groupby("subject")["distance"].describe(percentiles).T,
    ], axis=1)
).T

print("All Fixations:")
fixation_dist_summary

All Fixations:


,count,mean,std,min,5%,25%,50%,75%,90%,95%,max
all,151020.0,8.369685,5.141677,0.005305,0.889084,4.562637,7.852672,11.493848,15.068416,17.606240,44.068504
1,6253.0,8.613569,5.507893,0.016797,0.779704,4.677489,7.940661,11.820348,15.810998,18.558879,35.682843
2,14389.0,7.812967,4.818801,0.024893,0.737673,4.164936,7.490875,10.729949,14.051866,16.323540,30.079391
3,12377.0,8.286546,5.199138,0.005305,0.914627,4.541306,7.677497,11.189080,14.962309,17.784680,34.849555
12,12898.0,8.576142,5.210592,0.021815,1.032352,4.816429,8.010359,11.619241,15.205678,18.284497,31.039929
13,9435.0,8.422231,5.369897,0.041144,0.821337,4.469881,7.835926,11.482076,15.409945,18.367757,32.371653
14,12100.0,8.720280,5.235352,0.039167,1.124951,4.946421,8.067515,11.862541,15.509261,17.961658,44.068504
15,8334.0,8.083724,4.994525,0.049419,0.807365,4.141661,7.720790,11.402820,14.889734,16.939233,39.595361
16,11213.0,8.617470,4.926637,0.007104,1.066428,4.950468,8.219153,11.801094,15.180406,17.450055,41.260549
17,8754.0,7.985454,5.007073,0.015262,0.940709,4.236108,7.462323,11.073768,14.698249,16.915627,29.743946


##### find identification fixations
fixations where either:
- the subject performed an identification action during the fixation
- the subject performed an identification action immediately after the fixation

In [7]:
fixs_with_ident_time = fixations.copy()
fixs_with_ident_time["target"] = fixs_with_ident_time[dva_cols].idxmin(axis=1).str.replace("_distance_dva", "")
fixs_with_ident_time["distance_dva"] = fixs_with_ident_time[dva_cols].min(axis=1)
fixs_with_ident_time = (
    fixs_with_ident_time
    .drop(columns=[col for col in fixs_with_ident_time.columns if "_distance_" in col])
    .merge(
        idents.loc[
            idents["identification_category"] == "hit", ["subject", "trial", "target", "time"]
        ], on=["subject", "trial", "target"], how="left"
    )
)

fixs_with_ident_time.loc[:, "is_during"] = (fixs_with_ident_time["start_time"] <= fixs_with_ident_time["time"]) & (fixs_with_ident_time["time"] <= fixs_with_ident_time["end_time"])

fixs_with_ident_time.loc[:, "end_to_ident_diff"] = fixs_with_ident_time["time"] - fixs_with_ident_time["end_time"]
fixs_with_ident_time.loc[:, "is_immediately_preceding"] = False
immediately_preceding_idxs = (
    fixs_with_ident_time
    .loc[(0 <= fixs_with_ident_time["end_to_ident_diff"]) & (fixs_with_ident_time["end_to_ident_diff"] <= 1000)]    # max 1 sec
    .groupby(["subject", "trial", "eye", "target"])["end_to_ident_diff"]
    .idxmin()
    .values
)
fixs_with_ident_time.loc[immediately_preceding_idxs, "is_immediately_preceding"] = True
# fixs_with_ident_time.drop(columns=["end_to_ident_diff"], inplace=True)

In [8]:
ident_fixs = fixs_with_ident_time.loc[fixs_with_ident_time["is_during"]]
ident_fixs_dist_summary = (
    pd.concat([
        ident_fixs["distance_dva"].describe(percentiles).rename("all"),
        ident_fixs.groupby("subject")["distance_dva"].describe(percentiles).T,
    ], axis=1)
).T

print("Identification Fixations:")
ident_fixs_dist_summary

Identification Fixations:


,count,mean,std,min,5%,25%,50%,75%,90%,95%,max
all,2339.0,0.494843,0.273384,0.006736,0.114746,0.293956,0.454416,0.660410,0.872416,0.971007,2.338724
1,115.0,0.557825,0.266739,0.044864,0.167699,0.357158,0.561075,0.719662,0.879358,0.977707,1.329456
2,160.0,0.528795,0.287386,0.038549,0.131430,0.289217,0.483125,0.760696,0.941329,1.006351,1.297531
3,198.0,0.442791,0.226669,0.014672,0.107188,0.298885,0.407806,0.554803,0.731696,0.870882,1.513686
12,136.0,0.527300,0.262565,0.021815,0.142911,0.335189,0.495572,0.708934,0.906566,0.972123,1.217615
13,171.0,0.530232,0.271627,0.041144,0.149586,0.346984,0.478144,0.673151,0.907194,1.056385,1.344511
14,183.0,0.391573,0.214237,0.042179,0.106609,0.222227,0.367571,0.530472,0.679740,0.753424,1.366193
15,160.0,0.580076,0.253730,0.049419,0.207063,0.381133,0.567302,0.782218,0.919227,0.969219,1.108418
16,180.0,0.392194,0.219451,0.039398,0.082794,0.215285,0.348203,0.565713,0.710165,0.766271,0.974541
17,165.0,0.519842,0.237701,0.027277,0.180251,0.328877,0.511808,0.670824,0.823805,0.927252,1.121995


In [9]:
preceding_fixs = fixs_with_ident_time.loc[fixs_with_ident_time["is_immediately_preceding"]]
preceding_fixs_dist_summary = (
    pd.concat([
        preceding_fixs["distance_dva"].describe(percentiles).rename("all"),
        preceding_fixs.groupby("subject")["distance_dva"].describe(percentiles).T,
    ], axis=1)
).T

print("Preceding Identification Fixations:")
preceding_fixs_dist_summary

Preceding Identification Fixations:


,count,mean,std,min,5%,25%,50%,75%,90%,95%,max
all,2334.0,1.287609,1.738013,0.007757,0.204948,0.491474,0.846157,1.400626,2.378414,3.838304,19.686305
1,132.0,0.979136,0.754028,0.062451,0.262296,0.481610,0.738491,1.168130,2.195696,2.670861,3.509158
2,165.0,1.389632,1.805703,0.038814,0.199154,0.459916,0.786553,1.363825,2.585812,5.342408,10.295021
3,198.0,1.694982,2.225540,0.026879,0.167814,0.512888,0.921018,1.726912,4.324278,5.232185,14.438551
12,133.0,1.534387,1.712426,0.028722,0.347126,0.596772,1.033246,1.668116,2.593659,4.868113,9.493544
13,170.0,0.939942,0.639076,0.057448,0.229837,0.523218,0.813986,1.229925,1.600396,1.973387,5.260598
14,171.0,1.595634,2.510545,0.039167,0.252553,0.498334,0.872662,1.684271,3.360087,4.483183,16.502108
15,159.0,1.782779,2.845992,0.076795,0.222818,0.540849,0.883613,1.885028,3.485988,5.057811,16.967226
16,172.0,1.499751,2.320020,0.092160,0.211473,0.549600,0.929410,1.464096,2.931335,4.588137,19.686305
17,166.0,1.776691,1.956814,0.076016,0.319646,0.700914,1.168373,1.915614,3.789113,7.027576,10.451534


In [11]:
column_titles = ["All Fixations", "Co-Occurring with Identification", "Preceding Identification"]
fig = make_subplots(
    rows=2, cols=len(column_titles), column_titles=column_titles,
    shared_xaxes=True, shared_yaxes=False,
)

for c in range(len(column_titles)):
    if c == 0:
        data = fixs_with_ident_time
    elif c == 1:
        data = fixs_with_ident_time[fixs_with_ident_time["is_during"]]
    elif c == 2:
        data = fixs_with_ident_time[fixs_with_ident_time["is_immediately_preceding"]]
    else:
        raise ValueError(f"Unexpected column index {c}.")
    # top: distribution across all subjects
    fig.add_trace(
        row=1, col=c+1, trace=go.Violin(
            y0="distance", x=data["distance_dva"],
            name="All Subjects", legendgroup="All Subjects",
            text=data.apply(
                lambda row: f"Subject: {row['subject']}<br>"
                            f"Trial: {row['trial']}<br>"
                            f"Target: {row['target']}<br>"
                            f"Distance: {row['distance_dva']:.2f} DVA",
                axis=1
            ),
            marker=dict(color=cnfg.get_discrete_color("all")),
            width=1.75, orientation="h", side="positive", spanmode='hard',
            box=dict(visible=False),
            meanline=dict(visible=True),
            points="all", pointpos=-0.5,
            showlegend=c==0, hoverinfo="x+y+text",

        )
    )
    # bottom: distribution per subject
    for subj_id in data[cnfg.SUBJECT_STR].unique():
        subj_string = f"{cnfg.SUBJECT_STR.capitalize()} {subj_id:02d}"
        subj_data = data[data[cnfg.SUBJECT_STR] == subj_id]
        texts = subj_data.apply(
            lambda row: f"{subj_string}<br>"
                        f"Trial: {row['trial']}<br>"
                        f"Target: {row['target']}<br>"
                        f"Distance: {row['distance_dva']:.2f} DVA",
            axis=1
        )
        fig.add_trace(
            row=2, col=c+1, trace=go.Violin(
                y0="distance", x=subj_data["distance_dva"],
                text=texts,
                name=subj_string, legendgroup=subj_string,
                marker=dict(color=cnfg.get_discrete_color(subj_id, loop=True), opacity=0.5),
                width=1.75, orientation="h", side="positive", spanmode='hard',
                box=dict(visible=False),
                meanline=dict(visible=True),
                points="all", pointpos=-0.5,
                showlegend=c==0, hoverinfo="x+y+text"
            )
        )

# update visuals
fig.update_annotations(font=cnfg.AXIS_LABEL_FONT)
fig.update_yaxes(showticklabels=False)  # Hide y-axis labels
fig.update_xaxes(
    title=None, showline=False,
    showgrid=True, gridcolor=cnfg.GRID_LINE_COLOR, gridwidth=cnfg.GRID_LINE_WIDTH,
    zeroline=False, zerolinecolor=cnfg.GRID_LINE_COLOR, zerolinewidth=cnfg.ZERO_LINE_WIDTH,
    tickfont=cnfg.AXIS_TICK_FONT,
)
fig.update_layout(
    width=1400, height=650,
    title=dict(text="Distance on Fixations", font=cnfg.TITLE_FONT),
    paper_bgcolor='rgba(0, 0, 0, 0)',
    # plot_bgcolor='rgba(0, 0, 0, 0)',
    showlegend=True,
)

fig.show()